In [23]:
import torch
import numpy as np
import pandas as pd
import os
from RBM import RBM
#from load_dataset import MNIST
from tqdm import trange

In [33]:
def initialize_model():
	model = torch.nn.Sequential(
		torch.nn.Linear(20, 1600),
		torch.nn.Sigmoid(),
		torch.nn.Linear(1600, 8),
		torch.nn.Softmax(dim=1),
	)
	return model


In [38]:
def generate_batches(x, y, batch_size=64):
	x = x[:int(x.shape[0] - x.shape[0]%batch_size)]
	x = torch.reshape(x, (x.shape[0]//batch_size, batch_size, x.shape[1]))
	y = y[:int(y.shape[0] - y.shape[0]%batch_size)]
	y = torch.reshape(y, (y.shape[0]//batch_size, batch_size))
	return {'x':x, 'y':y}


In [35]:
def test(model, train_x, train_y, test_x, test_y, epoch):
	criterion = torch.nn.CrossEntropyLoss()

	output_test = model(test_x)
	loss_test = criterion(output_test, test_y).item()
	output_test = torch.argmax(output_test, axis=1)
	acc_test = torch.sum(output_test == test_y).item()/test_y.shape[0]

	output_train = model(train_x)
	loss_train = criterion(output_train, train_y).item()
	output_train = torch.argmax(output_train, axis=1)
	acc_train = torch.sum(output_train == train_y).item()/train_y.shape[0]

	return epoch, loss_test, loss_train, acc_test, acc_train


In [62]:
def train(model, x, y, train_x, train_y, test_x, test_y, epochs=200):
	dataset = generate_batches(x, y)

	criterion = torch.nn.CrossEntropyLoss()
	optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
	training = trange(epochs)
	progress = []
	for epoch in training:
		running_loss = 0
		acc = 0
		for batch_x, target in zip(dataset['x'], dataset['y']):
			output = model(batch_x)
			loss = criterion(output, target)
			output = torch.argmax(output, dim=1)
			acc += torch.sum(output == target).item()/target.shape[0]
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()
			running_loss += loss.item()
		running_loss /= len(dataset['y'])
		acc /= len(dataset['y'])
		progress.append(test(model, train_x, train_y, test_x, test_y, epoch+1))
		training.set_description(str({'epoch': epoch+1, 'loss': round(running_loss, 4), 'acc': round(acc, 4)}))

	return model, progress


In [6]:
import glob
import os
#from sklearn.decomposition import PCA, KernelPCA

from torch.utils.data import TensorDataset, DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch import nn
from torch import optim
import torchvision.transforms as tr
import matplotlib.pyplot as plt

import time

In [7]:
path = 'C:/Users/kbj/Downloads/data/train/x/'
data_train_x_list = os.listdir('C:/Users/kbj/Downloads/data/train/x/')
path = 'C:/Users/kbj/Downloads/data/train/x/'
x_train_mean = np.array([])
x_train_var = np.array([])
x_train_std = np.array([])
x_train_max = np.array([])
x_train_min = np.array([])

for i in data_train_x_list:
    df = pd.read_csv(path+i, header=None)
    mean = np.mean(df.to_numpy())
    var = np.var(df.to_numpy())
    std = np.std(df.to_numpy())
    max = np.max(df.to_numpy())
    min = np.min(df.to_numpy())
    x_train_mean = np.append(x_train_mean, mean)
    x_train_var = np.append(x_train_var, var)
    x_train_std = np.append(x_train_std, std)
    x_train_max = np.append(x_train_max, max)
    x_train_min = np.append(x_train_min, min)

x_train_feature = np.concatenate((x_train_mean.reshape(-1,1), x_train_var.reshape(-1,1), x_train_std.reshape(-1,1), x_train_max.reshape(-1,1), x_train_min.reshape(-1,1)), axis=1)
x_train_feature.shape

path = 'C:/Users/kbj/Downloads/data/train/y/'
data_train_y_list = os.listdir('C:/Users/kbj/Downloads/data/train/y/')
path = 'C:/Users/kbj/Downloads/data/train/y/'
y_train_mean = np.array([])
y_train_var = np.array([])
y_train_std = np.array([])
y_train_max = np.array([])
y_train_min = np.array([])

for i in data_train_y_list:
    df = pd.read_csv(path+i, header=None)
    mean = np.mean(df.to_numpy())
    var = np.var(df.to_numpy())
    std = np.std(df.to_numpy())
    max = np.max(df.to_numpy())
    min = np.min(df.to_numpy())
    y_train_mean = np.append(y_train_mean, mean)
    y_train_var = np.append(y_train_var, var)
    y_train_std = np.append(y_train_std, std)
    y_train_max = np.append(y_train_max, max)
    y_train_min = np.append(y_train_min, min)

y_train_feature = np.concatenate((y_train_mean.reshape(-1,1), y_train_var.reshape(-1,1), y_train_std.reshape(-1,1), y_train_max.reshape(-1,1), y_train_min.reshape(-1,1)), axis=1)
y_train_feature.shape

path = 'C:/Users/kbj/Downloads/data/train/z/'
data_train_z_list = os.listdir('C:/Users/kbj/Downloads/data/train/z/')
path = 'C:/Users/kbj/Downloads/data/train/z/'
z_train_mean = np.array([])
z_train_var = np.array([])
z_train_std = np.array([])
z_train_max = np.array([])
z_train_min = np.array([])

for i in data_train_z_list:
    df = pd.read_csv(path+i, header=None)
    mean = np.mean(df.to_numpy())
    var = np.var(df.to_numpy())
    std = np.std(df.to_numpy())
    max = np.max(df.to_numpy())
    min = np.min(df.to_numpy())
    z_train_mean = np.append(z_train_mean, mean)
    z_train_var = np.append(z_train_var, var)
    z_train_std = np.append(z_train_std, std)
    z_train_max = np.append(z_train_max, max)
    z_train_min = np.append(z_train_min, min)

z_train_feature = np.concatenate((z_train_mean.reshape(-1,1), z_train_var.reshape(-1,1), z_train_std.reshape(-1,1), z_train_max.reshape(-1,1), z_train_min.reshape(-1,1)), axis=1)
z_train_feature.shape

path = 'C:/Users/kbj/Downloads/data/train/e/'
data_train_e_list = os.listdir('C:/Users/kbj/Downloads/data/train/e/')
path = 'C:/Users/kbj/Downloads/data/train/e/'
e_train_mean = np.array([])
e_train_var = np.array([])
e_train_std = np.array([])
e_train_max = np.array([])
e_train_min = np.array([])

for i in data_train_e_list:
    df = pd.read_csv(path+i, header=None)
    mean = np.mean(df.to_numpy())
    var = np.var(df.to_numpy())
    std = np.std(df.to_numpy())
    max = np.max(df.to_numpy())
    min = np.min(df.to_numpy())
    e_train_mean = np.append(e_train_mean, mean)
    e_train_var = np.append(e_train_var, var)
    e_train_std = np.append(e_train_std, std)
    e_train_max = np.append(e_train_max, max)
    e_train_min = np.append(e_train_min, min)

e_train_feature = np.concatenate((e_train_mean.reshape(-1,1), e_train_var.reshape(-1,1), e_train_std.reshape(-1,1), e_train_max.reshape(-1,1), e_train_min.reshape(-1,1)), axis=1)
e_train_feature.shape

#xyz_train_feature = np.sqrt(np.square(x_train_feature[:,0])+np.square(y_train_feature[:,0])+np.square(z_train_feature[:,0]))
#xyz_train_feature = xyz_train_feature.reshape(-1,1)
#all_train_feature = np.concatenate((x_train_feature, y_train_feature, z_train_feature, e_train_feature, xyz_train_feature), axis=1)
all_train_feature = np.concatenate((x_train_feature, y_train_feature, z_train_feature, e_train_feature), axis=1)
all_train_feature.shape

train_label = pd.read_csv('C:/Users/kbj/Downloads/data/train/train_label.csv', header=None)
train_label = train_label.to_numpy()

train_label.shape

(700, 1)

In [8]:
path_x = 'C:/Users/kbj/Downloads/data/test/x/'
data_test_x_list = os.listdir('C:/Users/kbj/Downloads/data/test/x/')
x_test_mean = np.array([])
x_test_var = np.array([])
x_test_std = np.array([])
x_test_max = np.array([])
x_test_min = np.array([])

for i in data_test_x_list:
    df = pd.read_csv(path_x+i, header=None)
    mean = np.mean(df.to_numpy())
    var = np.var(df.to_numpy())
    std = np.std(df.to_numpy())
    max = np.max(df.to_numpy())
    min = np.min(df.to_numpy())
    x_test_mean = np.append(x_test_mean, mean)
    x_test_var = np.append(x_test_var, var)
    x_test_std = np.append(x_test_std, std)
    x_test_max = np.append(x_test_max, max)
    x_test_min = np.append(x_test_min, min)

x_test_feature = np.concatenate((x_test_mean.reshape(-1,1), x_test_var.reshape(-1,1), x_test_std.reshape(-1,1), x_test_max.reshape(-1,1), x_test_min.reshape(-1,1)), axis=1)
print(x_test_feature.shape)

path_y = 'C:/Users/kbj/Downloads/data/test/y/'
data_test_y_list = os.listdir('C:/Users/kbj/Downloads/data/test/y/')
y_test_mean = np.array([])
y_test_var = np.array([])
y_test_std = np.array([])
y_test_max = np.array([])
y_test_min = np.array([])

for i in data_test_y_list:
    df = pd.read_csv(path_y+i, header=None)
    mean = np.mean(df.to_numpy())
    var = np.var(df.to_numpy())
    std = np.std(df.to_numpy())
    max = np.max(df.to_numpy())
    min = np.min(df.to_numpy())
    y_test_mean = np.append(y_test_mean, mean)
    y_test_var = np.append(y_test_var, var)
    y_test_std = np.append(y_test_std, std)
    y_test_max = np.append(y_test_max, max)
    y_test_min = np.append(y_test_min, min)

y_test_feature = np.concatenate((y_test_mean.reshape(-1,1), y_test_var.reshape(-1,1), y_test_std.reshape(-1,1), y_test_max.reshape(-1,1), y_test_min.reshape(-1,1)), axis=1)
print(y_test_feature.shape)

path_z = 'C:/Users/kbj/Downloads/data/test/z/'
data_test_z_list = os.listdir('C:/Users/kbj/Downloads/data/test/z/')
z_test_mean = np.array([])
z_test_var = np.array([])
z_test_std = np.array([])
z_test_max = np.array([])
z_test_min = np.array([])

for i in data_test_z_list:
    df = pd.read_csv(path_z+i, header=None)
    mean = np.mean(df.to_numpy())
    var = np.var(df.to_numpy())
    std = np.std(df.to_numpy())
    max = np.max(df.to_numpy())
    min = np.min(df.to_numpy())
    z_test_mean = np.append(z_test_mean, mean)
    z_test_var = np.append(z_test_var, var)
    z_test_std = np.append(z_test_std, std)
    z_test_max = np.append(z_test_max, max)
    z_test_min = np.append(z_test_min, min)

z_test_feature = np.concatenate((z_test_mean.reshape(-1,1), z_test_var.reshape(-1,1), z_test_std.reshape(-1,1), z_test_max.reshape(-1,1), z_test_min.reshape(-1,1)), axis=1)
print(z_test_feature.shape)

path_e = 'C:/Users/kbj/Downloads/data/test/e/'
data_test_e_list = os.listdir('C:/Users/kbj/Downloads/data/test/e/')
e_test_mean = np.array([])
e_test_var = np.array([])
e_test_std = np.array([])
e_test_max = np.array([])
e_test_min = np.array([])

for i in data_test_e_list:
    df = pd.read_csv(path_e+i, header=None)
    mean = np.mean(df.to_numpy())
    var = np.var(df.to_numpy())
    std = np.std(df.to_numpy())
    max = np.max(df.to_numpy())
    min = np.min(df.to_numpy())
    e_test_mean = np.append(e_test_mean, mean)
    e_test_var = np.append(e_test_var, var)
    e_test_std = np.append(e_test_std, std)
    e_test_max = np.append(e_test_max, max)
    e_test_min = np.append(e_test_min, min)

e_test_feature = np.concatenate((e_test_mean.reshape(-1,1), e_test_var.reshape(-1,1), e_test_std.reshape(-1,1), e_test_max.reshape(-1,1), e_test_min.reshape(-1,1)), axis=1)
print(e_test_feature.shape)

#xyz_test_feature = np.sqrt(np.square(x_test_feature[:,0])+np.square(y_test_feature[:,0])+np.square(z_test_feature[:,0]))
#xyz_test_feature = xyz_test_feature.reshape(-1,1)
#all_test_feature = np.concatenate((x_test_feature, y_test_feature, z_test_feature, e_test_feature, xyz_test_feature), axis=1)

all_test_feature = np.concatenate((x_test_feature, y_test_feature, z_test_feature, e_test_feature), axis=1)
print(all_test_feature.shape)

test_label = pd.read_csv('C:/Users/kbj/Downloads/data/test/test_label.csv', header=None)
test_label = test_label.to_numpy()

print(test_label.shape)

(1400, 5)
(1400, 5)
(1400, 5)
(1400, 5)
(1400, 20)
(1400, 1)


In [9]:
print(all_train_feature.shape)
print(train_label.shape)

print(all_test_feature.shape)
print(test_label.shape)

(700, 20)
(700, 1)
(1400, 20)
(1400, 1)


In [63]:
from sklearn.preprocessing import StandardScaler

x_train = all_train_feature
y_train = train_label.reshape(-1)
x_test = all_test_feature
y_test = test_label.reshape(-1)

sc = StandardScaler()
sc.fit(x_train)

x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [64]:
train_x = torch.Tensor(x_train_std)
train_y = torch.LongTensor(y_train)
test_x = torch.Tensor(x_test_std)
test_y = torch.LongTensor(y_test)

ds_train = TensorDataset(train_x, train_y)
ds_test = TensorDataset(test_x, test_y)

loader_train = DataLoader(ds_train, batch_size=128, shuffle=True)
loader_test = DataLoader(ds_test, shuffle=False)

In [65]:
if __name__ == '__main__':
	#mnist = MNIST()
	#train_x, train_y, test_x, test_y = mnist.load_dataset()
	print('MAE for all 0 selection:', torch.mean(train_x))
	vn = train_x.shape[1]
	hn = 1600

	rbm = RBM(vn, hn, epochs=100, mode='bernoulli', lr=0.0001, k=1, batch_size=64, gpu=True, optimizer='adam', savefile='phm_trained_rbm.pt', early_stopping_patience=1000)
	rbm.train(train_x)

{'epoch': 1, 'loss': 0.6821}:   1%|          | 1/100 [00:00<00:11,  8.87it/s]

MAE for all 0 selection: tensor(4.0872e-10)


{'epoch': 100, 'loss': 0.6401}: 100%|██████████| 100/100 [00:09<00:00, 11.03it/s]


In [66]:
if __name__ == '__main__':
	#mnist = MNIST()
	#train_x, train_y, test_x, test_y = mnist.load_dataset()
	
	vn = train_x.shape[1]
	hn = 1600
	rbm = RBM(vn, hn)
	rbm.load_rbm('phm_trained_rbm.pt')

	model = initialize_model()

	model, progress = train(model, train_x, train_y, train_x, train_y, test_x, test_y)
	progress = pd.DataFrame(np.array(progress))
	progress.columns = ['epochs', 'test loss', 'train loss', 'test acc', 'train acc']
	progress.to_csv('RBM_without_pretraining_classifier.csv', index=False)

	model = initialize_model()

	model[0].weight = torch.nn.Parameter(rbm.W)
	model[0].bias = torch.nn.Parameter(rbm.hb)

	model, progress = train(model, train_x, train_y, train_x, train_y, test_x, test_y)
	progress = pd.DataFrame(np.array(progress))
	progress.columns = ['epochs', 'test loss', 'train loss', 'test acc', 'train acc']
	progress.to_csv('RBM_pretrained_classifier.csv', index=False)

{'epoch': 200, 'loss': 1.4772, 'acc': 0.8562}: 100%|██████████| 200/200 [00:18<00:00, 10.63it/s]
